In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%load_ext autoreload
%autoreload 2

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '3'

import sys

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import logging


logger = logging.getLogger('sequence_tagger_bert')

formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

logger.handlers = []

fhandler = logging.handlers.TimedRotatingFileHandler(filename='logs.txt', when='midnight')
fhandler.setFormatter(formatter)
logger.addHandler(fhandler)

handler = logging.StreamHandler(sys.stdout)
handler.setFormatter(formatter)
logger.addHandler(handler)

logger.setLevel(logging.DEBUG)

In [4]:
import torch

device = torch.device('cuda')
n_gpu = torch.cuda.device_count()

for i in range(n_gpu):
    print(torch.cuda.get_device_name(i))

Tesla V100-DGXS-32GB


In [5]:
CACHE_DIR = '../workdir/cache'
BATCH_SIZE = 16
#BATCH_SIZE = 8
PRED_BATCH_SIZE = 1000
MAX_LEN = 128
MAX_N_EPOCHS = 10
#MAX_N_EPOCHS = 100
#MAX_N_EPOCHS = 50
#MAX_N_EPOCHS = 10
REDUCE_ON_PLATEAU = False
WEIGHT_DECAY = 0.01
LEARNING_RATE = 3e-6
#LEARNING_RATE = 1e-5
#LEARNING_RATE = 2e-5

In [6]:
!pip3 install flair

/bin/sh: 1: pip3: not found


In [14]:
!locale

LANG=C.UTF-8
LANGUAGE=
LC_CTYPE="C.UTF-8"
LC_NUMERIC="C.UTF-8"
LC_TIME="C.UTF-8"
LC_COLLATE="C.UTF-8"
LC_MONETARY="C.UTF-8"
LC_MESSAGES="C.UTF-8"
LC_PAPER="C.UTF-8"
LC_NAME="C.UTF-8"
LC_ADDRESS="C.UTF-8"
LC_TELEPHONE="C.UTF-8"
LC_MEASUREMENT="C.UTF-8"
LC_IDENTIFICATION="C.UTF-8"
LC_ALL=


In [13]:
!export LC_ALL=en_US.UTF-8

In [11]:
from flair.datasets import ColumnCorpus


data_folder = '/workspace/bert_sequence_tagger/src/data/NER/Varvara_v3'
corpus = ColumnCorpus(data_folder, 
                      {0 : 'text', 1 : 'tag'},
                      train_file='/home/vika/targer/data/JUNE/train_pred_full_positive.tsv',
                      test_file='/home/vika/targer/data/JUNE/dev_pred_full_positive.tsv',
                      dev_file='/home/vika/targer/data/JUNE/dev_pred_full_positive.tsv')

print(corpus.obtain_statistics())

I0610 23:45:15.415585 140316096190208 file_utils.py:39] PyTorch version 1.3.1 available.
/home/user/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/user/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/user/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/us

I0610 23:45:17.491529 140316096190208 textcleaner.py:37] 'pattern' package not found; tag filters are not available for English


2020-06-10 23:45:17,571 Reading data from /workspace/bert_sequence_tagger/src/data/NER/Varvara_v3
2020-06-10 23:45:17,572 Train: /home/vika/targer/data/JUNE/train_pred_full_positive.tsv
2020-06-10 23:45:17,572 Dev: /home/vika/targer/data/JUNE/dev_pred_full_positive.tsv
2020-06-10 23:45:17,573 Test: /home/vika/targer/data/JUNE/dev_pred_full_positive.tsv


UnicodeDecodeError: 'utf-8' codec can't decode bytes in position 1692-1693: invalid continuation byte

In [9]:
!printenv LANG

C.UTF-8


In [52]:
a = corpus.make_tag_dictionary(tag_type = 'tag')

In [53]:
a.idx2item

[b'<unk>',
 b'O',
 b'B-OBJ',
 b'B-PREDFULL',
 b'I-PREDFULL',
 b'I-OBJ',
 b'<START>',
 b'<STOP>']

In [10]:
from bert_sequence_tagger import SequenceTaggerBert, BertForTokenClassificationCustom
from pytorch_transformers import BertTokenizer, BertForTokenClassification
import torch.nn as nn

from bert_sequence_tagger.bert_utils import make_bert_tag_dict_from_flair_corpus


bpe_tokenizer = BertTokenizer.from_pretrained('bert-base-cased', cache_dir=None, do_lower_case=False)

idx2tag, tag2idx = make_bert_tag_dict_from_flair_corpus(corpus)

model = nn.DataParallel(BertForTokenClassificationCustom.from_pretrained('bert-base-cased', cache_dir=None, num_labels=len(tag2idx))).cuda()
#model = BertForTokenClassification.from_pretrained('bert-base-cased', cache_dir=CACHE_DIR, num_labels=len(tag2idx)).cuda()

seq_tagger = SequenceTaggerBert(bert_model=model, bpe_tokenizer=bpe_tokenizer, idx2tag=idx2tag, tag2idx=tag2idx, max_len=200)

I0610 23:28:15.503033 140016840734464 modeling_bert.py:226] Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .
I0610 23:28:15.510476 140016840734464 modeling_xlnet.py:339] Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .
I0610 23:30:24.735812 140016840734464 file_utils.py:234] https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-vocab.txt not found in cache, downloading to /tmp/tmpi_n5u6hv
E0610 23:32:35.806068 140016840734464 tokenization_utils.py:289] Couldn't reach server to download vocabulary.


NameError: name 'corpus' is not defined

In [55]:
bpe_tokenizer

In [56]:
BATCH_SIZE = 10
PRED_BATCH_SIZE = 10
MAX_N_EPOCHS = 20
from torch.utils.data import RandomSampler, SequentialSampler

from bert_sequence_tagger.bert_utils import create_loader_from_flair_corpus, get_parameters_without_decay
from bert_sequence_tagger.model_trainer_bert import ModelTrainerBert

from pytorch_transformers import AdamW, WarmupLinearSchedule

from bert_sequence_tagger.metrics import f1_entity_level, f1_token_level

test_dataloader = create_loader_from_flair_corpus(corpus.test,
                                                  SequentialSampler,
                                                  batch_size=PRED_BATCH_SIZE)

_, __, test_metrics = seq_tagger.predict(test_dataloader, evaluate=True, 
                                         metrics=[f1_entity_level, f1_token_level])
logger.info(f'Entity-level f1: {test_metrics[1]}')
logger.info(f'Token-level f1: {test_metrics[2]}')

train_dataloader = create_loader_from_flair_corpus(corpus.train, 
                                                   RandomSampler, 
                                                   batch_size=BATCH_SIZE)
val_dataloader = create_loader_from_flair_corpus(corpus.dev,
                                                 SequentialSampler,
                                                 batch_size=PRED_BATCH_SIZE)

optimizer = AdamW(get_parameters_without_decay(model), lr=LEARNING_RATE, betas=(0.9, 0.999), 
                  eps =1e-6, weight_decay=0.01, correct_bias=True)
lr_scheduler = WarmupLinearSchedule(optimizer, warmup_steps=0.1, 
                                    t_total=(len(corpus.train) / BATCH_SIZE)*MAX_N_EPOCHS)
trainer = ModelTrainerBert(model=seq_tagger, 
                           optimizer=optimizer, 
                           lr_scheduler=lr_scheduler,
                           train_dataloader=train_dataloader, 
                           val_dataloader=val_dataloader,
                           update_scheduler='es',
                           keep_best_model=True,
                           restore_bm_on_lr_change=False,
                           max_grad_norm=1.,
                           validation_metrics=[f1_entity_level],
                           decision_metric=lambda metrics: -metrics[1])

trainer.train(epochs=MAX_N_EPOCHS)


test_dataloader = create_loader_from_flair_corpus(corpus.test,
                                                  SequentialSampler,
                                                  batch_size=PRED_BATCH_SIZE)

_, __, test_metrics = seq_tagger.predict(test_dataloader, evaluate=True, 
                                         metrics=[f1_entity_level, f1_token_level])
logger.info(f'Entity-level f1: {test_metrics[1]}')
logger.info(f'Token-level f1: {test_metrics[2]}')

2019-11-20 00:18:43,851 - sequence_tagger_bert - INFO - Entity-level f1: 0.032533889468196034
2019-11-20 00:18:43,852 - sequence_tagger_bert - INFO - Token-level f1: 0.06207475067573866


Epoch:   0%|          | 0/20 [00:00<?, ?it/s]

2019-11-20 00:19:36,479 - sequence_tagger_bert - INFO - Train loss: 0.3619607299953312
2019-11-20 00:19:39,953 - sequence_tagger_bert - INFO - Validation loss: 0.3088280222401386
2019-11-20 00:19:39,954 - sequence_tagger_bert - INFO - Validation metrics: (0.3985239852398525,)
2019-11-20 00:19:39,973 - sequence_tagger_bert - INFO - Current learning rate: 0.0


Epoch:   5%|▌         | 1/20 [00:56<17:44, 56.04s/it]

2019-11-20 00:20:32,119 - sequence_tagger_bert - INFO - Train loss: 0.08740380431174652
2019-11-20 00:20:34,944 - sequence_tagger_bert - INFO - Validation loss: 0.39300361003090695
2019-11-20 00:20:34,945 - sequence_tagger_bert - INFO - Validation metrics: (0.4347826086956522,)
2019-11-20 00:20:34,964 - sequence_tagger_bert - INFO - Current learning rate: 0.0


...

2019-11-20 00:36:01,028 - sequence_tagger_bert - INFO - Train loss: 0.018303427412839874
2019-11-20 00:36:03,860 - sequence_tagger_bert - INFO - Validation loss: 0.5431420835098479
2019-11-20 00:36:03,991 - sequence_tagger_bert - INFO - Validation metrics: (0.47466007416563666,)
2019-11-20 00:36:03,992 - sequence_tagger_bert - INFO - Current learning rate: 0.0


Epoch:  95%|█████████▌| 19/20 [17:20<00:54, 54.21s/it]

2019-11-20 00:36:54,500 - sequence_tagger_bert - INFO - Train loss: 0.018760964634991554
2019-11-20 00:36:57,514 - sequence_tagger_bert - INFO - Validation loss: 0.5384372797817355
2019-11-20 00:36:57,515 - sequence_tagger_bert - INFO - Validation metrics: (0.4773006134969324,)
2019-11-20 00:36:57,534 - sequence_tagger_bert - INFO - Current learning rate: 0.0


Epoch: 100%|██████████| 20/20 [18:13<00:00, 54.01s/it]


2019-11-20 00:37:00,991 - sequence_tagger_bert - INFO - Entity-level f1: 0.34105653382761814
2019-11-20 00:37:00,992 - sequence_tagger_bert - INFO - Token-level f1: 0.3320134023758757


In [57]:
with open('labels_v3_predful_manual.txt', 'w') as f:
    for _string in _:
        #f.seek(0)
        f.write(', '.join(_string) + '\n')

In [ ]:
def prepare_flair_corpus(corpus, name='tag', filter_tokens={'-DOCSTART-'}):
    result = []
    for sent in corpus[:10]:
        print ("sent", sent)
        print ("sent[0].text", sent[0].text)
        if sent[0].text in filter_tokens:
            continue
        else:
            result.append(([token.text for token in sent.tokens],
                           [token.tags[name].value for token in sent.tokens]))
    
    return result


In [ ]:
test_dataloader = create_loader_from_flair_corpus(corpus.test,
                                                  SequentialSampler,
                                                  batch_size=PRED_BATCH_SIZE)

_, __, test_metrics = seq_tagger.predict(test_dataloader, evaluate=True, 
                                         metrics=[f1_entity_level, f1_token_level])
logger.info(f'Entity-level f1: {test_metrics[1]}')
logger.info(f'Token-level f1: {test_metrics[2]}')

In [ ]:
(0.9143007822800387, 0.9306361914074436)